In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
from astropy.table import Table
import numpy as np
import os.path
import datetime 

In [2]:
from surveysim.wrapper import surveySim

In [3]:
def collect_tile_id(start, end):
    day_now = start
    id_list = np.empty((0), dtype=int)
    id_plan = np.empty((0), dtype=int)
    while(day_now <= end):
        day_now = day_now + datetime.timedelta(days=1)
        listdirName = "{}{}{:02d}{:02d}{}".format('obslist',day_now.year, day_now.month, day_now.day, '.fits')   
        plandirName = "{}{}{:02d}{:02d}{}".format('obsplan',day_now.year, day_now.month, day_now.day, '.fits')    

        #print(day_now,listdirName)
        if os.path.exists(listdirName):
            a = Table.read(listdirName)
            set_a = set(a['TILEID'])
            set_id = set(id_list)
            set_diff = set_a - set_id
            id_list = np.append(id_list, np.array(list(set_diff), dtype=int)) #only add the IDs that don't repeat
        
        if os.path.exists(plandirName):
            a = Table.read(plandirName)
            set_a = set(a['TILEID'])
            set_id = set(id_plan)
            set_diff = set_a - set_id
            id_plan = np.append(id_plan, np.array(list(set_diff), dtype=int)) #only add the IDs that don't repeat
    
    return {'OBSERVED':id_list, 'PLANNED':id_plan}

In [5]:
! rm -f tiles_observed.fits
start_string = "01-12-2017"
start = datetime.datetime.strptime(start_string, "%d-%m-%Y")

n_epochs = 3
days_per_epoch = 4

fiberassign_cadence = datetime.timedelta(days=days_per_epoch)

for i in range(n_epochs):
    print('\n\n')
    surveySim(start.day, start.month, start.year, end.day, end.month, end.year)
    #dd = collect_tile_id(start, end)
    #print dd['OBSERVED'], len(dd['OBSERVED']), len(dd['PLANNED'])
    #print ('These were the observed tiles: {}'.format(dd['OBSERVED']))
    #np.savetxt('observed_epoch{}.txt'.format(i), dd['OBSERVED'], fmt='%d')
    #np.savetxt('epoch{}.txt'.format(i), dd['PLANNED'], fmt='%d')
    start = start + fiberassign_cadence 
    end = start + fiberassign_cadence




The survey will start from scratch.
Conditions at the beginning of the night: 
{'Seeing': 1.0784803045489388, 'Clouds': 0.19273237986273045, 'Transparency': 0.4144720277433709, 'OpenDome': True}


IERSRangeError: (some) times are outside of range covered by IERS table.

In [ ]:
#for i in range(n_epochs):
#    id_plan = np.loadtxt("epoch{}.txt".format(i))
#    id_list = np.loadtxt("observed_epoch{}.txt".format(i))
#    print("epoch {}".format(i))
#    print("planned {}".format(len(id_plan), id_plan))
#    print("done {} {}\n\n".format(len(id_list), id_list))

done_tiles = Table.read("tiles_observed.fits")
print done_tiles
set(done_tiles['TILEID']), set(done_tiles['STATUS'])